In [1]:
import requests
import pandas as pd
from config import census_key
import json




In [6]:
# Creates dataframe of school districts, states, and census codes
name = ['New York City Department of Education',
        'Chicago Public School District',
        'Los Angeles Unified School District',
        'Houston Independent School District',
        'Cave Creek Unified District',
        'Deer Valley Unified District',
        'Glendale Union High School District',
        'Phoenix Union High School District',
        'Tempe Union High School District',
       'Tolleson Union High School District',
        'Paradise Valley Unified District',
        'Scottsdale Unified District',
       'Dallas Independent School District',
       'San Antonio Independent School District',
       'Philadelphia City School District',
        'San Diego City Unified School District',
        'San Jose Unified School District'
       ]
state = ['36','17','06',
         '48','04','04',
         '04','04','04',
         '04','04','04',
         '48','48','42',
         '06','06'
        ]
# The value for Cave Creek Unified District is incorrect due to unexpected error. Do not use values returned for Cave Creek
district = ['20580','09930','22710',
            '23640','07750','07750',
            '03450','06330','08340',
            '08520','05930','07570',
            '16230','38730','18990',
            '34320', '34590'
           ]

full_code = ['9700000US3620580','9700000US1709930','9700000US0622710',
             '9700000US4823640','9700000US0400001','9700000US0407750',
             '9600000US0403450','9600000US0406330', '9600000US0408340',
             '9600000US0408520','9700000US0405930', '9700000US0407570',
             '9700000US4816230','9700000US4838730','9700000US4218990',
             '9700000US0634320','9700000US0634590'
            ]
districts_df = pd.DataFrame(name)
districts_df['State'] = state
districts_df['District'] = district
districts_df['Full_Code'] = full_code
districts_df.columns = ['Name', 'State', 'District', 'Full_Code']
# Unified and secondary need slightly different API requests
districts_df['Unified_Secondary'] = ['U','U','U',
                                     'U','U','U',
                                     'S','S','S',
                                     'S','U','U',
                                    'U','U','U',
                                    'U','U']
districts_df17 = districts_df
districts_df18 = districts_df
districts_df18

,Name,State,District,Full_Code,Unified_Secondary
0,New York City Department of Education,36,20580,9700000US3620580,U
1,Chicago Public School District,17,09930,9700000US1709930,U
2,Los Angeles Unified School District,06,22710,9700000US0622710,U
3,Houston Independent School District,48,23640,9700000US4823640,U
4,Cave Creek Unified District,04,07750,9700000US0400001,U
5,Deer Valley Unified District,04,07750,9700000US0407750,U
6,Glendale Union High School District,04,03450,9600000US0403450,S
7,Phoenix Union High School District,04,06330,9600000US0406330,S
8,Tempe Union High School District,04,08340,9600000US0408340,S
9,Tolleson Union High School District,04,08520,9600000US0408520,S


In [104]:
# API call loop for table B14005 2018
# some setup before the loop
#set base url for 2018
base_url = 'https://api.census.gov/data/2018/acs/acs1?'
group = "B14005"
district = ""
state = ""
selectB14005_full = []
# loop through each district in districts_df
for index, row in districts_df18.iterrows():
    district = row['District']
    state = row['State']
    u_s = row['Unified_Secondary']
    # Check for unified
    if u_s == 'U':
        try:
            print(f"Trying {row['Name']} with {district} and {state}")
            response = requests.get(f"{base_url}get=group({group})&for=school%20district%20(unified):{district}&in=state:{state}&key={census_key}").json()
        except:
            print(f" request failed with : {base_url}get=group({group})&for=school%20district%20(unified):{district}&in=state:{state}&key={census_key}") 
    # Check for secondary
    elif u_s == 'S':
        try:
            print(f"Trying {row['Name']} with {district} and {state}")
            response = requests.get(f"{base_url}get=group({group})&for=school%20district%20(secondary):{district}&in=state:{state}&key={census_key}").json()
        except:
            print(f" request failed with : {base_url}get=group({group})&for=school%20district%20(secondary):{district}&in=state:{state}&key={census_key}")
    # Clean call response
    response_df = pd.DataFrame(response)
    response_df.columns = cps_df.iloc[0]
    # trim columns
    selectB14005 = pd.DataFrame(response_df.iloc[1][['GEO_ID','state','school district (unified)',
                    'B14005_001E','B14005_002E','B14005_003E','B14005_004E',
                     'B14005_005E','B14005_006E','B14005_016E','B14005_017E',
                    'B14005_018E','B14005_019E', 'B14005_020E']])
    select2B14005 = selectB14005.transpose()
    # add to districts_df18
    selectB14005_full.append(select2B14005.iloc[0])
    
selectB14005_full_df = pd.DataFrame(selectB14005_full)
districts_df18B14005 = districts_df18.merge(selectB14005_full_df, left_on = 'Full_Code', right_on = 'GEO_ID')
districts_df18B14005

Trying New York City Department of Education with 20580 and 36
Trying Chicago Public School District with 09930 and 17
Trying Los Angeles Unified School District with 22710 and 06
Trying Houston Independent School District with 23640 and 48
Trying Cave Creek Unified District with 00001 and 04
 request failed with : https://api.census.gov/data/2018/acs/acs1?get=group(B14005)&for=school%20district%20(unified):00001&in=state:04&key=9c5aa4a34753830c81ff60db8991b08b03d03dd5
Trying Deer Valley Unified District with 07750 and 04
Trying Glendale Union High School District with 03450 and 04
Trying Phoenix Union High School District with 06330 and 04
Trying Tempe Union High School District with 08340 and 04
Trying Tolleson Union High School District with 08520 and 04
Trying Paradise Valley Unified District with 05930 and 04
Trying Scottsdale Unified District with 07570 and 04
Trying Dallas Independent School District with 16230 and 48
Trying San Antonio Independent School District with 38730 and

,Name,State,District,Full_Code,Unified_Secondary,GEO_ID,state,school district (unified),B14005_001E,B14005_002E,B14005_003E,B14005_004E,B14005_005E,B14005_006E,B14005_016E,B14005_017E,B14005_018E,B14005_019E,B14005_020E
0,New York City Department of Education,36,20580,9700000US3620580,U,9700000US3620580,36,20580,357275,177894,156235,16652,5923,133660,179381,160744,24066,9942,126736
1,Chicago Public School District,17,09930,9700000US1709930,U,9700000US1709930,17,09930,122213,59551,50420,7315,3673,39432,62662,56451,11212,3021,42218
2,Los Angeles Unified School District,06,22710,9700000US0622710,U,9700000US0622710,06,22710,233195,118023,99451,15546,6122,77783,115172,103278,17060,5070,81148
3,Houston Independent School District,48,23640,9700000US4823640,U,9700000US4823640,48,23640,75320,39622,31518,3597,1858,26063,35698,30848,4953,1752,24143
4,Houston Independent School District,48,23640,9700000US4823640,U,9700000US4823640,48,23640,75320,39622,31518,3597,1858,26063,35698,30848,4953,1752,24143
5,Deer Valley Unified District,04,07750,9700000US0407750,U,9700000US0407750,04,07750,None,None,None,None,None,None,None,None,None,None,None
6,Glendale Union High School District,04,03450,9600000US0403450,S,9600000US0403450,04,03450,17310,7922,6218,1480,248,4490,9388,6415,1748,722,3945
7,Phoenix Union High School District,04,06330,9600000US0406330,S,9600000US0406330,04,06330,52802,24668,18832,4874,912,13046,28134,22927,5895,742,16290
8,Tempe Union High School District,04,08340,9600000US0408340,S,9600000US0408340,04,08340,None,None,None,None,None,None,None,None,None,None,None
9,Tolleson Union High School District,04,08520,9600000US0408520,S,9600000US0408520,04,08520,None,None,None,None,None,None,None,None,None,None,None


In [11]:
# API call loop for table B23002A-B23002I 2018 which give employment by age group for ethnic groups
# some setup before the loop
#set base url for 2018
base_url = 'https://api.census.gov/data/2018/acs/acs1?'


group_list = ["B23002B", "B23002C", "B23002D", "B23002E",
              "B23002F", "B23002G", "B23002H", "B23002I"]
district = ""
state = ""
selectB23002_full = []
# loop through each district in districts_df
for index, row in districts_df18.iterrows():
    district = row['District']
    state = row['State']
    u_s = row['Unified_Secondary']
    # loop through each table in group list
    for i in group_list:
        # Check for unified school district
        if u_s == 'U':
            try:
                print(f"Trying {row['Name']} with {district} and {state}")
                response = requests.get(f"{base_url}get=group({i})&for=school%20district%20(unified):{district}&in=state:{state}&key={census_key}").json()
            except:
                print(f" request failed with : {base_url}get=group({i})&for=school%20district%20(unified):{district}&in=state:{state}&key={census_key}") 
        # Check for secondary district
        elif u_s == 'S':
            # Skipping secondary districts. After testing, some of the secondary districts do not have any data for 16-19 age group.
            # The missing columns in the response creates the error below.
            try:
                print(f"Skipping {row['Name']} on {i}")
                #print(f"Trying {row['Name']} with {district} and {state}")
                #response = requests.get(f"{base_url}get=group({i})&for=school%20district%20(secondary):{district}&in=state:{state}&key={census_key}").json()
                continue
            except:
                print(f" request failed with : {base_url}get=group({i})&for=school%20district%20(secondary):{district}&in=state:{state}&key={census_key}")
        # Clean call response
        response_df = pd.DataFrame(response)
        #print(response_df)
        response_df.columns = response_df.iloc[0]
        # trim columns
        # set up column names based on current table from group list
        selectcolumns = ['GEO_ID','state','school district (unified)',
                        i+'_003E',i+'_004E',
                     i+'_005E',i+'_006E',i+'_007E',i+'_008E',
                    i+'_009E',i+'_042E', i+'_043E',i+'_044E',
                    i+'_045E',i+'_046E',i+'_047E',i+'_048E']
        # The next line currently causes an error. "Passing list-likes to .loc or [] with any missing labels is no longer supported"
        # The error is caused by the first Glendale Union High request. Error may be caused by secondary school request.
        # Testing needed
        selectB23002 = pd.DataFrame(response_df.iloc[1][selectcolumns])
        select2B23002 = selectB23002.transpose()
    
        # add to districts_df18
        selectB23002_full.append(select2B23002.iloc[0])
        #selectB23002_full_df = pd.DataFrame(selectB23002_full)
        #print(selectB23002_full_df)
        districts_df18B23002 = districts_df18.merge(selectB23002_full_df, left_on = 'Full_Code', right_on = 'GEO_ID')


Trying New York City Department of Education with 20580 and 36
0            GEO_ID state school district (unified) B23002B_003E B23002B_004E  \
1  9700000US3620580    36                     20580        50895         9124   

0 B23002B_005E B23002B_006E B23002B_007E B23002B_008E B23002B_009E  \
1            0         9124         5653         3471        41771   

0 B23002B_042E B23002B_043E B23002B_044E B23002B_045E B23002B_046E  \
1        49656        11557            0        11557         7877   

0 B23002B_047E B23002B_048E  
1         3680        38099  
Trying New York City Department of Education with 20580 and 36
             GEO_ID state school district (unified) B23002B_003E B23002B_004E  \
1  9700000US3620580    36                     20580        50895         9124   
1  9700000US3620580    36                     20580          NaN          NaN   

  B23002B_005E B23002B_006E B23002B_007E B23002B_008E B23002B_009E  ...  \
1            0         9124         5653         3

             GEO_ID state school district (unified) B23002B_003E B23002B_004E  \
1  9700000US3620580    36                     20580        50895         9124   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US3620580    36                     20580          NaN          NaN   

  B23002B_005E B23002B_006E B23002B_007E B23002B_008E B23002B_009E  ...  \
1            0         9124         5653         3471        41771  ...   
1          NaN          NaN          NaN          NaN          NaN  ...   
1          NaN          NaN          NaN          NaN          NaN  ...   
1          NaN          NaN          NaN          N

             GEO_ID state school district (unified) B23002B_003E B23002B_004E  \
1  9700000US3620580    36                     20580        50895         9124   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US1709930    17                     09930        19307         4187   
1  9700000US1709930    17                     09930          NaN          NaN   

  B23002B_005E B23002B_006E B23002B_007E B23002B_008E B23002B_009E  ...  \
1            0         9124      

             GEO_ID state school district (unified) B23002B_003E B23002B_004E  \
1  9700000US3620580    36                     20580        50895         9124   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US1709930    17                     09930        19307         4187   
1  9700000US1709930    17                     09930          NaN          NaN   
1  9700000US1709930    17                     09930          NaN          NaN   
1  9700000US1709930    17   

             GEO_ID state school district (unified) B23002B_003E B23002B_004E  \
1  9700000US3620580    36                     20580        50895         9124   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US3620580    36                     20580          NaN          NaN   
1  9700000US1709930    17                     09930        19307         4187   
1  9700000US1709930    17                     09930          NaN          NaN   
1  9700000US1709930    17                     09930          NaN          NaN   
1  9700000US1709930    17   

...

              GEO_ID state school district (unified) B23002B_003E  \
1   9700000US3620580    36                     20580        50895   
1   9700000US3620580    36                     20580          NaN   
1   9700000US3620580    36                     20580          NaN   
1   9700000US3620580    36                     20580          NaN   
1   9700000US3620580    36                     20580          NaN   
..               ...   ...                       ...          ...   
1   9700000US4218990    42                     18990          NaN   
1   9700000US4218990    42                     18990          NaN   
1   9700000US0634320    06                     34320         2473   
1   9700000US0634320    06                     34320          NaN   
1   9700000US0634320    06                     34320          NaN   

   B23002B_004E B23002B_005E B23002B_006E B23002B_007E B23002B_008E  \
1          9124            0         9124         5653         3471   
1           NaN          NaN 

              GEO_ID state school district (unified) B23002B_003E  \
1   9700000US3620580    36                     20580        50895   
1   9700000US3620580    36                     20580          NaN   
1   9700000US3620580    36                     20580          NaN   
1   9700000US3620580    36                     20580          NaN   
1   9700000US3620580    36                     20580          NaN   
..               ...   ...                       ...          ...   
1   9700000US0634320    06                     34320          NaN   
1   9700000US0634320    06                     34320          NaN   
1   9700000US0634320    06                     34320          NaN   
1   9700000US0634320    06                     34320          NaN   
1   9700000US0634320    06                     34320          NaN   

   B23002B_004E B23002B_005E B23002B_006E B23002B_007E B23002B_008E  \
1          9124            0         9124         5653         3471   
1           NaN          NaN 

              GEO_ID state school district (unified) B23002B_003E  \
1   9700000US3620580    36                     20580        50895   
1   9700000US3620580    36                     20580          NaN   
1   9700000US3620580    36                     20580          NaN   
1   9700000US3620580    36                     20580          NaN   
1   9700000US3620580    36                     20580          NaN   
..               ...   ...                       ...          ...   
1   9700000US0634320    06                     34320          NaN   
1   9700000US0634320    06                     34320          NaN   
1   9700000US0634320    06                     34320          NaN   
1   9700000US0634320    06                     34320          NaN   
1   9700000US0634590    06                     34590         None   

   B23002B_004E B23002B_005E B23002B_006E B23002B_007E B23002B_008E  \
1          9124            0         9124         5653         3471   
1           NaN          NaN 

              GEO_ID state school district (unified) B23002B_003E  \
1   9700000US3620580    36                     20580        50895   
1   9700000US3620580    36                     20580          NaN   
1   9700000US3620580    36                     20580          NaN   
1   9700000US3620580    36                     20580          NaN   
1   9700000US3620580    36                     20580          NaN   
..               ...   ...                       ...          ...   
1   9700000US0634320    06                     34320          NaN   
1   9700000US0634590    06                     34590         None   
1   9700000US0634590    06                     34590          NaN   
1   9700000US0634590    06                     34590          NaN   
1   9700000US0634590    06                     34590          NaN   

   B23002B_004E B23002B_005E B23002B_006E B23002B_007E B23002B_008E  \
1          9124            0         9124         5653         3471   
1           NaN          NaN 

              GEO_ID state school district (unified) B23002B_003E  \
1   9700000US3620580    36                     20580        50895   
1   9700000US3620580    36                     20580          NaN   
1   9700000US3620580    36                     20580          NaN   
1   9700000US3620580    36                     20580          NaN   
1   9700000US3620580    36                     20580          NaN   
..               ...   ...                       ...          ...   
1   9700000US0634590    06                     34590          NaN   
1   9700000US0634590    06                     34590          NaN   
1   9700000US0634590    06                     34590          NaN   
1   9700000US0634590    06                     34590          NaN   
1   9700000US0634590    06                     34590          NaN   

   B23002B_004E B23002B_005E B23002B_006E B23002B_007E B23002B_008E  \
1          9124            0         9124         5653         3471   
1           NaN          NaN 

In [15]:
districts_df18B23002_grouped = districts_df18B23002.groupby('Full_Code').first().reset_index()
districts_df18B23002_grouped

,Full_Code,Name,State,District,Unified_Secondary,GEO_ID,state,school district (unified),B23002B_003E,B23002B_004E,...,B23002I_007E,B23002I_008E,B23002I_009E,B23002I_042E,B23002I_043E,B23002I_044E,B23002I_045E,B23002I_046E,B23002I_047E,B23002I_048E
0,9700000US0405930,Paradise Valley Unified District,04,05930,U,9700000US0405930,04,05930,None,None,...,None,None,None,None,None,None,None,None,None,None
1,9700000US0407570,Scottsdale Unified District,04,07570,U,9700000US0407570,04,07570,None,None,...,None,None,None,None,None,None,None,None,None,None
2,9700000US0407750,Deer Valley Unified District,04,07750,U,9700000US0407750,04,07750,None,None,...,None,None,None,None,None,None,None,None,None,None
3,9700000US0622710,Los Angeles Unified School District,06,22710,U,9700000US0622710,06,22710,9363,3390,...,17378,5293,55541,75953,19132,207,18925,14440,4485,56821
4,9700000US0634320,San Diego City Unified School District,06,34320,U,9700000US0634320,06,34320,2473,537,...,1631,383,6560,9481,2423,137,2286,1841,445,7058
5,9700000US0634590,San Jose Unified School District,06,34590,U,9700000US0634590,06,34590,None,None,...,953,69,2188,3868,1428,0,1428,941,487,2440
6,9700000US1709930,Chicago Public School District,17,09930,U,9700000US1709930,17,09930,19307,4187,...,5399,1756,15983,22587,8104,0,8104,6757,1347,14483
7,9700000US3620580,New York City Department of Education,36,20580,U,9700000US3620580,36,20580,50895,9124,...,7620,4646,51018,64057,15286,0,15286,10995,4291,48771
8,9700000US4218990,Philadelphia City School District,42,18990,U,9700000US4218990,42,18990,16494,5429,...,1776,1292,4832,7110,1269,0,1269,1110,159,5841
9,9700000US4816230,Dallas Independent School District,48,16230,U,9700000US4816230,48,16230,5619,1260,...,4753,914,10779,17639,6101,0,6101,5452,649,11538


### The cells below are only for testing and troubleshooting. Remember to remove before final version

In [2]:
# This cell is just test calls
# API call reference for ACS 2018 1yr: https://api.census.gov/data/2018/acs/acs1.html

#examples found here: https://api.census.gov/data/2018/acs/acs1/examples.html
#set base url for 2018
base_url = 'https://api.census.gov/data/2018/acs/acs1?'
# setup employment data for Chicago Public Schools

# variable groups found here: https://api.census.gov/data/2018/acs/acs1/groups.html 
group = 'B14005'

# school district code found on map available here: https://www2.census.gov/geo/maps/dc10map/sch_dist/st17_il/c17031_cook/
# this method requires personally reading pdf maps to find code. need to find a bettter way
school_district = '09930'

# state code found here: https://www.census.gov/library/reference/code-lists/ansi/ansi-codes-for-states.html
state ='17'

# request it
chicago_response = requests.get(f"{base_url}get=group({group})&for=school%20district%20(unified):{school_district}&in=state:{state}&key={census_key}").json()
print(json.dumps(chicago_response, indent=4, sort_keys=True))


[
    [
        "GEO_ID",
        "B14005_001E",
        "B14005_001M",
        "B14005_002E",
        "B14005_002M",
        "B14005_003E",
        "B14005_003M",
        "B14005_004E",
        "B14005_004M",
        "B14005_005E",
        "B14005_005M",
        "B14005_006E",
        "B14005_006M",
        "B14005_007E",
        "B14005_007M",
        "B14005_008E",
        "B14005_008M",
        "B14005_009E",
        "B14005_009M",
        "B14005_010E",
        "B14005_010M",
        "B14005_011E",
        "B14005_011M",
        "B14005_012E",
        "B14005_012M",
        "B14005_013E",
        "B14005_013M",
        "B14005_014E",
        "B14005_014M",
        "B14005_015E",
        "B14005_015M",
        "B14005_016E",
        "B14005_016M",
        "B14005_017E",
        "B14005_017M",
        "B14005_018E",
        "B14005_018M",
        "B14005_019E",
        "B14005_019M",
        "B14005_020E",
        "B14005_020M",
        "B14005_021E",
        "B14005_021M",
        

In [3]:
# another test cell
cps_df = pd.DataFrame(chicago_response)
cps_df.columns = cps_df.iloc[0]
cps_df


,GEO_ID,B14005_001E,B14005_001M,B14005_002E,B14005_002M,B14005_003E,B14005_003M,B14005_004E,B14005_004M,B14005_005E,...,B14005_026MA,B14005_026EA,B14005_027MA,B14005_027EA,B14005_028MA,B14005_028EA,B14005_029MA,B14005_029EA,state,school district (unified)
0,GEO_ID,B14005_001E,B14005_001M,B14005_002E,B14005_002M,B14005_003E,B14005_003M,B14005_004E,B14005_004M,B14005_005E,...,B14005_026MA,B14005_026EA,B14005_027MA,B14005_027EA,B14005_028MA,B14005_028EA,B14005_029MA,B14005_029EA,state,school district (unified)
1,9700000US1709930,122213,5273,59551,3711,50420,3438,7315,1420,3673,...,None,None,None,None,None,None,None,None,17,09930


In [7]:
# Cleans B14005. Generally, this table covers enrollment and employment in 16-19 y/o
cps_select = pd.DataFrame(cps_df.iloc[1][['GEO_ID','state','school district (unified)',
                    'B14005_001E','B14005_002E','B14005_003E','B14005_004E',
                     'B14005_005E','B14005_006E','B14005_016E','B14005_017E',
                    'B14005_018E','B14005_019E', 'B14005_020E']])
cps_select2 = cps_select.transpose()
cps_select2

,GEO_ID,state,school district (unified),B14005_001E,B14005_002E,B14005_003E,B14005_004E,B14005_005E,B14005_006E,B14005_016E,B14005_017E,B14005_018E,B14005_019E,B14005_020E
1,9700000US1709930,17,09930,122213,59551,50420,7315,3673,39432,62662,56451,11212,3021,42218


In [20]:
# Get B23002A-B23002I which give employment by age group for ethnic groups

#set base url for 2018
base_url = 'https://api.census.gov/data/2018/acs/acs1?'
# setup employment data for Chicago Public Schools

# variable groups found here: https://api.census.gov/data/2018/acs/acs1/groups.html 
group = 'B23002B'

# school district code found on map available here: https://www2.census.gov/geo/maps/dc10map/sch_dist/st17_il/c17031_cook/
# this method requires personally reading pdf maps to find code. need to find a bettter way
school_district = '09930'

# state code found here: https://www.census.gov/library/reference/code-lists/ansi/ansi-codes-for-states.html
state ='17'

# request it
chicago_response = requests.get(f"{base_url}get=group({group})&for=school%20district%20(unified):{school_district}&in=state:{state}&key={census_key}").json()

cps_df_B23002B = pd.DataFrame(chicago_response)
cps_df_B23002B.columns = cps_df_B23002B.iloc[0]
cps_df_B23002B

,B23002B_013M,B23002B_014E,B23002B_014M,B23002B_015E,B23002B_015M,B23002B_016E,B23002B_016M,B23002B_017E,B23002B_017M,B23002B_018E,...,B23002B_009EA,B23002B_010MA,B23002B_010EA,B23002B_011MA,B23002B_011EA,B23002B_012MA,B23002B_012EA,B23002B_013EA,state,school district (unified)
0,B23002B_013M,B23002B_014E,B23002B_014M,B23002B_015E,B23002B_015M,B23002B_016E,B23002B_016M,B23002B_017E,B23002B_017M,B23002B_018E,...,B23002B_009EA,B23002B_010MA,B23002B_010EA,B23002B_011MA,B23002B_011EA,B23002B_012MA,B23002B_012EA,B23002B_013EA,state,school district (unified)
1,1837,14265,1935,4723,1350,9918,2098,144265,4669,111817,...,None,None,None,None,None,None,None,None,17,09930


In [21]:
# Cleans B23002B. Generally, this table covers employment by age group for african americans
cps_select_B23002B = pd.DataFrame(cps_df_B23002B.iloc[1][['GEO_ID','state','school district (unified)',
                    'B23002B_003E','B23002B_004E','B23002B_003E','B23002B_004E',
                     'B23002B_005E','B23002B_006E','B23002B_007E','B23002B_008E',
                    'B23002B_009E','B23002B_042E', 'B23002B_043E','B23002B_044E',
                    'B23002B_045E','B23002B_046E','B23002B_047E','B23002B_048E']])
cps_select_B23002B2 = cps_select_B23002B.transpose()
cps_select_B23002B2

,GEO_ID,state,school district (unified),B23002B_003E,B23002B_004E,B23002B_003E,B23002B_004E,B23002B_005E,B23002B_006E,B23002B_007E,B23002B_008E,B23002B_009E,B23002B_042E,B23002B_043E,B23002B_044E,B23002B_045E,B23002B_046E,B23002B_047E,B23002B_048E
1,9700000US1709930,17,09930,19307,4187,19307,4187,0,4187,1863,2324,15120,20919,5069,0,5069,3095,1974,15850


In [13]:
# Get annotation json for B14005

columnsB14005_json = requests.get('https://api.census.gov/data/2018/acs/acs1/groups/B14005.json').json()

print(json.dumps(columnsB14005_json, indent = 4, sort_keys= True))

{
    "variables": {
        "B14005_001E": {
            "concept": "SEX BY SCHOOL ENROLLMENT BY EDUCATIONAL ATTAINMENT BY EMPLOYMENT STATUS FOR THE POPULATION 16 TO 19 YEARS",
            "group": "B14005",
            "label": "Estimate!!Total",
            "limit": 0,
            "predicateOnly": true,
            "predicateType": "int"
        },
        "B14005_001EA": {
            "group": "B14005",
            "label": "Annotation of Estimate!!Total",
            "limit": 0,
            "predicateOnly": true,
            "predicateType": "string"
        },
        "B14005_001M": {
            "concept": "SEX BY SCHOOL ENROLLMENT BY EDUCATIONAL ATTAINMENT BY EMPLOYMENT STATUS FOR THE POPULATION 16 TO 19 YEARS",
            "group": "B14005",
            "label": "Margin of Error!!Total",
            "limit": 0,
            "predicateOnly": true,
            "predicateType": "int"
        },
        "B14005_001MA": {
            "group": "B14005",
            "label": "Annotati

In [14]:
#columnsB14005 = columnsB14005['variables']
colnameB14005 = columnsB14005_json['variables']
colnameB14005


{'B14005_008EA': {'label': 'Annotation of Estimate!!Total!!Male!!Not enrolled in school!!High school graduate (includes equivalency)',
  'predicateType': 'string',
  'group': 'B14005',
  'limit': 0,
  'predicateOnly': True},
 'B14005_008MA': {'label': 'Annotation of Margin of Error!!Total!!Male!!Not enrolled in school!!High school graduate (includes equivalency)',
  'predicateType': 'string',
  'group': 'B14005',
  'limit': 0,
  'predicateOnly': True},
 'B14005_009MA': {'label': 'Annotation of Margin of Error!!Total!!Male!!Not enrolled in school!!High school graduate (includes equivalency)!!Employed',
  'predicateType': 'string',
  'group': 'B14005',
  'limit': 0,
  'predicateOnly': True},
 'B14005_009EA': {'label': 'Annotation of Estimate!!Total!!Male!!Not enrolled in school!!High school graduate (includes equivalency)!!Employed',
  'predicateType': 'string',
  'group': 'B14005',
  'limit': 0,
  'predicateOnly': True},
 'B14005_006EA': {'label': 'Annotation of Estimate!!Total!!Male!!E

In [15]:
# Create dictionary to rename dataframe columns to something description

colname2B14005 = list(colnameB14005.keys())
colname2B14005
collabelB14005 = []

for i in colname2B14005:
    collabelB14005.append(colnameB14005[i]['label'])
    
collabelB14005
colrenameB14005 = {colname2B14005[i]:collabelB14005[i] for i in range(len(colname2B14005))}
colrenameB14005

{'B14005_008EA': 'Annotation of Estimate!!Total!!Male!!Not enrolled in school!!High school graduate (includes equivalency)',
 'B14005_008MA': 'Annotation of Margin of Error!!Total!!Male!!Not enrolled in school!!High school graduate (includes equivalency)',
 'B14005_009MA': 'Annotation of Margin of Error!!Total!!Male!!Not enrolled in school!!High school graduate (includes equivalency)!!Employed',
 'B14005_009EA': 'Annotation of Estimate!!Total!!Male!!Not enrolled in school!!High school graduate (includes equivalency)!!Employed',
 'B14005_006EA': 'Annotation of Estimate!!Total!!Male!!Enrolled in school!!Not in labor force',
 'B14005_006MA': 'Annotation of Margin of Error!!Total!!Male!!Enrolled in school!!Not in labor force',
 'B14005_007EA': 'Annotation of Estimate!!Total!!Male!!Not enrolled in school',
 'B14005_007MA': 'Annotation of Margin of Error!!Total!!Male!!Not enrolled in school',
 'B14005_004EA': 'Annotation of Estimate!!Total!!Male!!Enrolled in school!!Employed',
 'B14005_004MA

In [20]:
# Renames columns to default census labels
cps_select2.rename(columns = colrenameB14005, inplace = True)
cps_select2

,GEO_ID,state,school district (unified),Estimate!!Total,Estimate!!Total!!Male,Estimate!!Total!!Male!!Enrolled in school,Estimate!!Total!!Male!!Enrolled in school!!Employed,Estimate!!Total!!Male!!Enrolled in school!!Unemployed,Estimate!!Total!!Male!!Enrolled in school!!Not in labor force,Estimate!!Total!!Female,Estimate!!Total!!Female!!Enrolled in school,Estimate!!Total!!Female!!Enrolled in school!!Employed,Estimate!!Total!!Female!!Enrolled in school!!Unemployed,Estimate!!Total!!Female!!Enrolled in school!!Not in labor force
1,9700000US1709930,17,09930,122213,59551,50420,7315,3673,39432,62662,56451,11212,3021,42218


In [65]:
# API fail on Cave Creek. 
# https://api.census.gov/data/2018/acs/acs1?get=group(B14005)&for=school%20district%20(unified):00001
# &in=state:04&key=9c5aa4a34753830c81ff60db8991b08b03d03dd5
# returns 204 error. 'No Content'
# confirmed district code
# It seems Cave Creek school district does not have a 1-year estimate only 5-year
# chicago_response = requests.get(f"https://api.census.gov/data/2018/acs/acs5?get=group(B14005)&for=school%20district%20(unified):00001&in=state:04&key=9c5aa4a34753830c81ff60db8991b08b03d03dd5").json()
# chicago_response

# Issue identified. Group notified. Resolution coming soon.

[['GEO_ID',
  'B14005_001E',
  'B14005_001M',
  'B14005_002E',
  'B14005_002M',
  'B14005_003E',
  'B14005_003M',
  'B14005_004E',
  'B14005_004M',
  'B14005_005E',
  'B14005_005M',
  'B14005_006E',
  'B14005_006M',
  'B14005_007E',
  'B14005_007M',
  'B14005_008E',
  'B14005_008M',
  'B14005_009E',
  'B14005_009M',
  'B14005_010E',
  'B14005_010M',
  'B14005_011E',
  'B14005_011M',
  'B14005_012E',
  'B14005_012M',
  'B14005_013E',
  'B14005_013M',
  'B14005_014E',
  'B14005_014M',
  'B14005_015E',
  'B14005_015M',
  'B14005_016E',
  'B14005_016M',
  'B14005_017E',
  'B14005_017M',
  'B14005_018E',
  'B14005_018M',
  'B14005_019E',
  'B14005_019M',
  'B14005_020E',
  'B14005_020M',
  'B14005_021E',
  'B14005_021M',
  'B14005_022E',
  'B14005_022M',
  'B14005_023E',
  'B14005_023M',
  'B14005_024E',
  'B14005_024M',
  'B14005_025E',
  'B14005_025M',
  'B14005_026E',
  'B14005_026M',
  'B14005_027E',
  'B14005_027M',
  'B14005_028E',
  'B14005_028M',
  'B14005_029E',
  'B14005_029M',
  

In [25]:
test_glendale = requests.get(f"https://api.census.gov/data/2018/acs/acs1?get=group(B23002B)&for=school%20district%20(secondary):03450&in=state:04&key={census_key}").json()

testGD_df = pd.DataFrame(test_glendale)
#print(response_df)
testGD_df.columns = testGD_df.iloc[0]
# trim columns
# set up column names based on current table from group list
#selecttestGDcol = ['GEO_ID','state','school district (unified)',
#                    'B23002B_003E','B23002B_004E','B23002B_003E','B23002B_004E',
#                     'B23002B_005E','B23002B_006E','B23002B_007E','B23002B_008E',
#                    'B23002B_009E','B23002B_042E', 'B23002B_043E','B23002B_044E',
#                    'B23002B_045E','B23002B_046E','B23002B_047E','B23002B_048E']
# The next line currently causes an error. "Passing list-likes to .loc or [] with any missing labels is no longer supported"
# The error is caused by the first Glendale Union High request. Error may be caused by secondary school request.
# Testing needed
#selecttestGD = pd.DataFrame(response_df.iloc[1]['GEO_ID','state','school district (unified)',
#                    'B23002B_003E','B23002B_004E','B23002B_003E','B23002B_004E',
#                     'B23002B_005E','B23002B_006E','B23002B_007E','B23002B_008E',
#                    'B23002B_009E','B23002B_042E', 'B23002B_043E','B23002B_044E',
#                    'B23002B_045E','B23002B_046E','B23002B_047E','B23002B_048E'])
#select2testGD = selecttestGD.transpose()
#select2testGD
testcolumns = testGD_df.columns
testcolumns.sort_values()
testcolumns

Index(['B23002B_013M', 'B23002B_014E', 'B23002B_014M', 'B23002B_015E',
       'B23002B_015M', 'B23002B_016E', 'B23002B_016M', 'B23002B_017E',
       'B23002B_017M', 'B23002B_018E',
       ...
       'B23002B_009EA', 'B23002B_010MA', 'B23002B_010EA', 'B23002B_011MA',
       'B23002B_011EA', 'B23002B_012MA', 'B23002B_012EA', 'B23002B_013EA',
       'state', 'school district (secondary)'],
      dtype='object', name=0, length=320)

-- Notes from this test --
Good News:
    - The above call works
    - pd.DataFrame defaults to the desired row/column
To do:  
    - convert the existing labels (B14005_XXXX) into descriptive labels
    - find better way to get school district codes. There must be a better way than going through pdf maps
    - try Census wrapper library to see if it resolves district and state code issues
    - look into table S2301
    - figure out how we might use margin of error in analysis. If unneeded, maybe drop it
Big picture:
    - build loops to request all districts in each year
    - assemble district data into dataframe for each year
    
-- Notes on tables --
B23002A-B23002I give employment by age group for ethnic groups  
B14005 gives employment by enrollment for age groups  

    